In [18]:
from pandas.core.frame import DataFrame
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import os

# Importing Files

In [19]:
project_links_df = pd.read_excel('project_links.xlsx')
project_links_df.head()

,Project Link,Project Approval Date,Project Code,County,Project Name,Project Status,Open Action Items
0,https://data.archwaypartnership.uga.edu/projec...,NaN,H-217,Hart,Feasibility Study for Recreation Center at the...,Completed,NaN
1,https://data.archwaypartnership.uga.edu/node/3705,NaN,H-184,Hart,Hart Botanical Garden Brochure,Completed,NaN
2,https://data.archwaypartnership.uga.edu/projec...,NaN,H-232,Hart,Historic Downtown Hartwell Ambassador Project ...,Completed,NaN
3,https://data.archwaypartnership.uga.edu/projec...,NaN,CA-73,Candler,Historical Marker Project,Completed,NaN
4,https://data.archwaypartnership.uga.edu/projec...,NaN,HB-34,Habersham,"""Hidden Gems of Habersham"" Brochure Project",Cancelled,NaN


In [20]:
# creating a new column for the code in the project link

project_links_df['Project Link Code'] = project_links_df['Project Link'].apply(lambda x: x.split("/")[-1])
project_links_df.head()

,Project Link,Project Approval Date,Project Code,County,Project Name,Project Status,Open Action Items,Project Link Code
0,https://data.archwaypartnership.uga.edu/projec...,NaN,H-217,Hart,Feasibility Study for Recreation Center at the...,Completed,NaN,3858
1,https://data.archwaypartnership.uga.edu/node/3705,NaN,H-184,Hart,Hart Botanical Garden Brochure,Completed,NaN,3705
2,https://data.archwaypartnership.uga.edu/projec...,NaN,H-232,Hart,Historic Downtown Hartwell Ambassador Project ...,Completed,NaN,3944
3,https://data.archwaypartnership.uga.edu/projec...,NaN,CA-73,Candler,Historical Marker Project,Completed,NaN,3661
4,https://data.archwaypartnership.uga.edu/projec...,NaN,HB-34,Habersham,"""Hidden Gems of Habersham"" Brochure Project",Cancelled,NaN,3231


### Function to rename files

In [21]:
project_links_df['Project Link Code'].iloc[0:10]

0    3858
1    3705
2    3944
3    3661
4    3231
5    3154
6    3674
7    2651
8    2652
9    2653
Name: Project Link Code, dtype: object

In [22]:
search_url = "https://data.archwaypartnership.uga.edu/"

browser = webdriver.Chrome()

# Open browser to begin search
browser.get(search_url)

# Maximize the window
browser.maximize_window()

time.sleep(2)

In [23]:
# entering the username
input_username = browser.find_element(By.ID,'edit-name')
input_username.send_keys("bs83243")

time.sleep(1)

# entering the password
input_password = browser.find_element(By.ID,'edit-pass')
input_password.send_keys("BS@15july1997")


time.sleep(1)

# clicking the login button
button_login = browser.find_element(By.ID,'edit-submit')
button_login.click()
time.sleep(2)

In [24]:
def get_table(browser):
    table = browser.find_elements(By.TAG_NAME, 'table')
    # Deliverable table is the 4th table on the page
    if len(table) < 4:
        return None
    return table[3]

def get_rows(table):
    tbody = table.find_elements(By.TAG_NAME, 'tbody')[0]
    return tbody.find_elements(By.TAG_NAME, 'tr')

In [25]:
def check_next_page(browser):
    nav = browser.find_elements(By.CLASS_NAME, 'pager')
    if len(nav) > 0:
        nav_list = nav[0].find_elements(By.TAG_NAME, 'li')
        for li in nav_list:
            a_tag = li.find_elements(By.TAG_NAME, 'a')
            if a_tag and 'Go to next page' in a_tag[0].get_attribute('title'):
                a_tag[0].click()
                time.sleep(2)
                return True
    return False

In [27]:
def rename_files_in_row(row, deliverable_folder):
    data = row.find_elements(By.TAG_NAME, 'td')[0]
    a_tag = data.find_element(By.TAG_NAME, 'a')
    del_link = a_tag.get_attribute('href')

    old_file_name = del_link.split('/')[-1]
    new_file_name = data.text

    old_file_path = os.path.join(deliverable_folder, old_file_name)
    new_file_path = os.path.join(deliverable_folder, new_file_name)

    if os.path.isfile(old_file_path):
        # There are some files with duplicate names
        if os.path.isfile(new_file_path):
            new_file_path = f"{new_file_path}_1"
        os.rename(old_file_path, new_file_path)
        print(f'Renamed {old_file_name} to {new_file_name}')

In [28]:
def process_deliverables(browser, deliverable_folder):
    next_page_exists = True
    while next_page_exists:
        next_page_exists = False
        table = get_table(browser)
        if table is None:
            # No deliverables found
            break
        rows = get_rows(table)
        for row in rows:
            rename_files_in_row(row, deliverable_folder)
        next_page_exists = check_next_page(browser)

In [29]:
# Load the CSV file with your data
df = project_links_df

# Iterate over the rows in the dataframe
for index, row in df.iterrows():
    code = str(row['Project Link Code'])
    project_link = row['Project Link']
    print(f'Processing project link: {project_link}')
    browser.get(project_link)
    time.sleep(2)
    
    # deliverable_folder = f'../../Downloads/Test-Rename/{code}-deliverables'
    deliverable_folder = f'/Users/bs83243/Downloads/Test-Deliverables-Extracted/{code}-deliverables'
    process_deliverables(browser, deliverable_folder)

Processing project link: https://data.archwaypartnership.uga.edu/project/3858
Renamed kVhru6vMYjd1.pptx to Capstone Poster_HartCoRecSports_Ludwick.pptx
Renamed VXhf6inqg6MH.pdf to C-450 ROOF PLAN.pdf
Renamed hGepbGpMhP13.pdf to A-000 COST ESTIMATE.pdf
Renamed dp4k48gPVnVS.pdf to C-101 EXISTING TOPOGRAPHY.pdf
Renamed v99B6M9didsj.pdf to A-502 SE SW ISOMETRIC.pdf
Renamed KIVSnNMq2GAW.pdf to A-401 NORTH SOUTH ELEVATION VIEW.pdf
Renamed EGCqTo9fn8Hj.pdf to A-300 THIRD FLOOR PLAN.pdf
Renamed 5qn7KmjBdMfP.pdf to C-602 ES&PC INTERMEDIATE.pdf
Renamed LGnTKF2ldO4q.pdf to G-000 COVER SHEET (NEW).pdf
Renamed IlyBzMbXc8ck.pdf to C-200 SITE PLAN.pdf
Renamed sGHkc87HFzXo.pdf to C-650 NPDES FINAL.pdf
Renamed NUxQ0oxVFKTf.pdf to A-100 FIRST FLOOR PLAN.pdf
Renamed NXu1W2bxyog9.pdf to C-100 EXISTING CONDITIONS.pdf
Renamed X2H0FNFdY4Fo.pdf to C-150 DEMOLITION PLAN.pdf
Renamed U9kYOfe6LMhN.pdf to C-600 ES&PC LEGEND.pdf
Renamed HGbJFeTiYhZP.pdf to C-500 UTILITY PLAN.pdf
Renamed KekfD8uhVP9H.pdf to C-102 PR

KeyboardInterrupt: 